In [2]:
import os
import pandas as pd
import logging
import json
import glob
import xlrd
from pathlib import Path
from cmath import nan
from datetime import datetime

In [11]:
def get_data_files(path, extensions):
    # for file in os.listdir(path):
    for path, subdirs, files in os.walk(path):
        for file in files:
            print(os.path.join(path, file))
            if os.path.isfile(os.path.join(path, file)):
                if Path(file).suffix in extensions:
                    # files = glob.glob(path)
                    # max_file = max(files, key=os.path.getctime)
                    yield os.path.join(path, file)

def check_quality(file):
    # Iterate through files in directory
    if Path(file).suffix == ".xlsx":
        df_quality = pd.read_excel(io=file, sheet_name=0, header=0)
        df_quality.head()
        logging.info("Importing data from Excel file: " + file)
    elif Path(file).suffix == ".csv":
        df_quality = pd.read_csv(filepath_or_buffer=file, sep=";")
        logging.info("Importing data from CSV file: " + file)
    
    # Create the nested dictionary
    quality_result = {}
    quality_result["columns_names"] = {}
    quality_result["columns_types"] = {}
    quality_result["columns_nulls"] = {}
    quality_result["columns_distincts"] = {}
    quality_result["rows_count"] = {}

    # Columns references
    # TODO: Create a patter for all datasets
    columns_ref_dataset1 = ["userId","name","city","locations"]
    columns_ref_dataset2 = ["locationId","name","rating"]

    # Iterate through the dataset columns and check if they exists in the pattern
    # Plus, insert the results in the dictionary
    for col in df_quality.columns:
        if col in columns_ref_dataset1:
            quality_result["columns_names"][col] = True
        else:
            quality_result["columns_names"][col] = False
    
     # Iterate through the dataset columns and check their datatypes
    for col in df_quality.columns:
        quality_result["columns_types"][col] = str(df_quality[col].dtypes)
    
    # Check the null values of each column
    for col in df_quality.columns:
        quality_result["columns_nulls"][col] = df_quality[col].isnull()
    
    # Check the distinct values of each column
    for col in df_quality.columns:
        quality_result["columns_distincts"][col] = df_quality[col].nunique()

    # Get the row count of the DF
    quality_result["rows_count"] = len(df_quality)

    return quality_result


In [13]:
data_files = []

path_url = r"C:\Users\andre\Desktop\coding\datasets"

for file in get_data_files(path_url, [".csv", ".xlsx"]):
    data_files.append(file)
    result = check_quality(file)
    print(result)
    json_object = json.dumps(result, indent = 4, ensure_ascii=False)

    current_datetime = datetime.now()
    str_current_datetime = str(current_datetime)
    file_name = "json_data_"+str_current_datetime+".json"

    with open(file_name, "w") as outfile:
        outfile.write(json_object)

C:\Users\andre\Desktop\coding\datasets\locations.csv
{'columns_names': {'locationId,name,rating': False}, 'columns_types': {'locationId,name,rating': 'object'}, 'columns_nulls': {'locationId,name,rating': 0    False
1    False
2    False
3    False
4    False
5    False
6    False
Name: locationId,name,rating, dtype: bool}, 'columns_distincts': {'locationId,name,rating': 7}, 'rows_count': 7}


TypeError: Object of type Series is not JSON serializable